In [8]:
import os
from skimage import color, io, measure
from PIL import Image
from itertools import product
import sys
import re
import subprocess



"""
The program divides te image into smaller pieces, as the network was trained on smaller resolution pics. Currently it was trained in the 1864x1864 squares.
To detect the flakes each image is divided into a grid of 1864x1864 images, with the margins cut off
"""
print("pic division started...")
def tile_label(filename, dir_in, dir_out, d):                #Funtion to divide the image, takes the filename, path for load and save and d- the size of the gid image, in this case 1864
    name, ext = os.path.splitext(filename)
    img = Image.open(os.path.join(dir_in, filename))
    #img = color.rgb2gray(img)
    w, h = img.size
    grid = product(range(0, h-h%d+1, d), range(0, w-w%d+1, d))  
    for i, j in grid:
        box = (j, i, j+d, i+d)
        out = os.path.join(dir_out, f'{name}_{i}_{j}{ext}')       
        img.crop(box).save(out)                              #cropping the box out of the original image
       
                                                            
image_set = io.imread_collection('*.jpg')                    #collecting all images into a single list                                      
for i, image in enumerate(image_set):    
    tile_label(image_set.files[i][:-4]+'.jpg',"","",1856)
    
print("pic division finished.")



"""
To detect images we used a yolov5n NN. This is the command that activates the detect.py script to detect flakes on the images cut out from the original one above.
"""

print("detection started...")
detect = "python3 ../insectRecognition/yolov5/detect.py "                                                        #yolov5 script for object detection
#size = "--img 1864 "                                                                                             #the size of the image on which detection is performedimage size
size = "--img 1856 "  
#weights = "--weights ../insectRecognition/yolov5/Big_Vlad_test_day10/yolov5n_size18645/weights/best.pt "         #a path to the weights file, based on training
weights = "--weights ../insectRecognition/yolov5/Big_Vlad_test_day22/yolov5n_size1856/weights/best.pt "
source = "--source " + os.getcwd()                                                                               #a source of files on which the detection is to be performed (in this case it's a folder)

os.system(detect+size+weights+source)                                                                            #detect flakes on each image
print("detection finished.")



"""
The last thing to do is to stitch the image with detected flakes back together, so it can be compared with the original one for an easy localization of the flakes
"""
print("pic stitching started...")
all_subdirs = [os.path.join("../insectRecognition/yolov5/runs/detect/",d) for d in os.listdir('../insectRecognition/yolov5/runs/detect/.') ]
latest_subdir = max(all_subdirs, key=os.path.getmtime)+"/"
grid_collection = []
image_set = io.imread_collection(latest_subdir+"*")
for i, image in enumerate(image_set):
    image_name = re.split("_",image_set.files[i][:-4])
    if len(image_name) == 1:
        new_image_name = image_name[0]       
        if grid_collection != []:            
            for image in grid_collection:                
                stitched_image.paste(Image.fromarray(image[0]),(int(image[1][2]),int(image[1][1])))
                delete_path = image[1][0].replace(latest_subdir,"")+"_"+image[1][1]+"_"+image[1][2]+".jpg"
                os.remove(delete_path)
            save_path = image[1][0].replace(latest_subdir,"")
            stitched_image.save(save_path+"_detected.jpg")
        grid_collection = []
        n = 0
        size_determined = 0
    else:
        grid_collection.append([image,image_name])
        if size_determined != True:
            size = image_name[1]
            size_determined = 1
        if image_name[1] == size:
            n = n+1
        else:
            img = Image.fromarray(image)
            stitched_image_width = n*img.width
            stitched_image_height = n*img.height
            stitched_image = Image.new("RGB", (stitched_image_width, stitched_image_height))
            img.close()
if grid_collection != []:
    for image in grid_collection:                
        stitched_image.paste(Image.fromarray(image[0]),(int(image[1][2]),int(image[1][1])))
        delete_path = image[1][0].replace(latest_subdir,"")+"_"+image[1][1]+"_"+image[1][2]+".jpg"
        os.remove(delete_path)
    save_path = image[1][0].replace(latest_subdir,"")
    stitched_image.save(save_path+"_detected.jpg")
print("pic stitching finished")



pic division started...
pic division finished.
detection started...


detect: weights=['../insectRecognition/yolov5/Big_Vlad_test_day22/yolov5n_size1856/weights/best.pt'], source=/home/cra9hack/Dropbox/PyProj/Untitled_Folder, data=../insectRecognition/yolov5/data/coco128.yaml, imgsz=[1856, 1856], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../insectRecognition/yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-208-g8236d881 Python-3.10.6 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7948MiB)

Fusing layers... 
Model summary: 157 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs
image 1/86 /home/cra9hack/Dropbox/PyProj/Untitled_Folder/img0.jpg: 1856x1664 (no detections), 10.8ms
image 2/86 /home/cra9hack/Dropbox/PyProj/Untitled_Folder/img0_0_0.jpg: 1856x1856 (no detection

detection finished.
pic stitching started...
pic stitching finished
